In [1]:
#!pip list #To obtain requirements.txt for Python 3 ipykernel (Python version 3.10.15)

In [2]:

# Notebook Summary:

# V.History: 
# Date Last Modified: 15 May 2025


#--------------------------------------------------------------------------------------------------
'''
    This notebook identifies the correlation for the target feature, Orthophosphate (previously 
    this was aimed for Phosphate), using an output file from the 
    notebook 08_nb_transpose_2959_all2.ipynb. 
    A PCA study was conducted to determine if these thousands of WQ parameters can be 
    represented by a reduced set of features.
'''
#--------------------------------------------------------------------------------------------------


#Pre-Requisite : 
    #Kernel Python 3 (ipykernel) is required to run this notebook 
    #Required python version - Python 3.10.15 and its compatible Numpy , ScikitLearn libraries

#Old Name: 14_NB_6Correlation_2959_All.ipynb


'\n    This notebook identifies the correlation for the target feature, Orthophosphate (previously \n    this was aimed for Phosphate), using an output file from the \n    notebook 08_nb_transpose_2959_all2.ipynb. \n    A PCA study was conducted to determine if these thousands of WQ parameters can be \n    represented by a reduced set of features.\n'

In [3]:
#Check python version compatibility 3.10 or above is required
!python -V
python_version=!(python --version 2>&1)
print (python_version)

Python 3.10.15
['Python 3.10.15']


In [4]:
ls ~/.local/share/jupyter/kernels

chem2/


In [5]:
#Check python version compatibility 3.10 or above is required

#sudo nano  ~/.local/share/jupyter/kernels/python311/kernel.json
#to display the system version of the pandas
#!pip show pandas
#to display this notebook's kernel version of the pandas
#%pip show pandas

import sys
sys.version

'3.10.15 | packaged by conda-forge | (main, Oct 16 2024, 01:24:24) [GCC 13.3.0]'

In [6]:
####################################################################################################
# Begin CARD
# Correlation Coefficient for OrthoPhosphate using all P-Family WQ Parameters of 
# 24 Years of England Data. Later this study was conducted for all 3000+ WQ Parameters based 
# on the feedback from Domain SME's from Xylem at the Cambridge Meet
####################################################################################################

In [7]:
%run "..//99_Common_Utils/99_NB_CommonUtils.ipynb" #Library Declaration section - Installing or Initiating all required Python Libraries

Intalling required libraries and utilities.....
Uses Python 3 (ipykernel) (Local)
Python 3.10.15
['Python 3.10.15']

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip in

2025-05-15 21:28:22.458385: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747344502.482645  180268 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747344502.490307  180268 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1747344502.514468  180268 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1747344502.514493  180268 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1747344502.514495  180268 computation_placer.cc:177] computation placer alr


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
|| Completed intalling required libraries and utilities ||

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [8]:
#Import PCA & Scaler Libraries
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [9]:
#STEP 1: Find Correlation % of P-Family parameters vs the target parameter Phosphate / Orthophosphate
#====================================================================================================

In [10]:
'''
There are 52 individual units of measure in the WQ monitoring data and over 38 for the P-Family. Phosphate or Orthophosphate is 
measured in 'mg/l'. Considerations were made to convert these metrics into a single unit of 'mg/l' (aligned to P unit measure). 
Below are the 'Conversion functions' / code snippets for that. However, the conversion was later discarded in favour of relative weights for 
different metrics in spatio-temporal models. The code is kept here to document this experiment.

Few notes about the considerations for the conversion given :

1. Density Considerations: Some conversions, such as % w/w or mg/kg, assume the substance has a similar density to water (1 g/ml). 
If the density differs, adjust the conversion factor accordingly.

2. Context-Specific Units: Units like bq/l (radioactivity), cfu/0.1l (colony-forming units), us/cm (conductivity), 
and rfu (fluorescence) require additional context or calibration to convert meaningfully to mg/l.

3. Load Units: Units like kg/ann, t/d, and t/qtr are load measurements and depend on the total volume of water per time period for conversion.

The code provides a basic framework, but real-world applications may require more specific formulas and data, especially for 
context-specific units.


There are 38 Unique metric for Phosphate family
mg/l , coded, ug/l , % , mg/kg, ug/kg, m, ppm, ngr, unitless, phi, ng/l, min, deg, no/g, kg/d, 
deccafix, pres/nf, mw.s/sq. , no. , no/100ml, abs/cm, nm, bq/l, text, % v/v, kt/qtr, t/qtr, kg/qtr,
t/d, % w/w, yes/no, l/kg/h, cfu/0.1l, us/cm, g/l, t/wk, ug
'''

#-------------------------------------------------------------------------------------
#SAMPLE CONVERSION :-
# Uni directional data sample 
#Sl.No. | Determinand_name | Determinand_Definition | Result | Date
#  1      Oxygen             Disolved oxygen           5.2     1-1-2000
#  2      Nitrogen           Nitrogen oxyde            4.1     1-1-2000
#  3      Oxygen             Disolved oxygen           5.8     2-1-2000
#  4      Nitrogen           Nitrogen oxyde            4.9     2-1-2000
#  5      Oxygen             Disolved oxygen           5.8     1-1-2000
#  6      Nitrogen           Nitrogen oxyde            4.9     1-1-2000

# Converted 2 dimesional data Sample 
#Sl.No. | Oxygen | Nitrogen | Date
#  1      11.0        4.1     1-1-2000
#  1      9.0         4.1     1-1-2000
#-------------------------------------------------------------------------------------


'''
Metrics conversion
'''


'\nMetrics conversion\n'

In [11]:
# Metrics Conversion functions
def first_convert_to_mg_per_l(value, unit, density=1.0):
    """
    Convert various units to mg/l.
    
    Parameters:
    - value: The numerical value to convert.
    - unit: The unit of the given value as a string.
    - density: Density of the substance in g/ml (default is 1.0, equivalent to water at room temperature).

    Returns:
    - The converted value in mg/l.
    """
    cUnit = 'mg/l'
    if unit == "mg/l":
        return value, cUnit
    elif unit == "%":
        # 1% = 10,000 mg/l
        return value * 10000, cUnit
    elif unit == "ug/l":
        # 1 ug/l = 0.001 mg/l
        return value * 0.001, cUnit
    elif unit == "mg/kg":
        # Assuming water density, 1 mg/kg = 1 mg/l
        return value, cUnit
    elif unit == "% v/v":
        # 1% v/v = 10,000 mg/l (for liquids like ethanol in water)
        return value * 10000, cUnit
    elif unit == "% w/w":
        # 1% w/w = 10,000 mg/l (assuming density is similar to water)
        return value * 10000, cUnit
    elif unit == "bq/l":
        # Conversion depends on the specific isotope (radioactivity)
        #raise ValueError("Conversion for 'bq/l' requires context-specific information.")
        return value, cUnit
    elif unit == "cfu/0.1l":
        # Colony forming units conversion needs microbiological context
        #raise ValueError("Conversion for 'cfu/0.1l' requires context-specific information.")
        return value, unit
    elif unit == "g/l":
        # 1 g/l = 1000 mg/l
        return value * 1000, cUnit
    elif unit == "kg/ann":
        # Annual load conversion requires volume flow data
        #raise ValueError("Conversion for 'kg/ann' requires annual volume flow data.")
        return value, unit
    elif unit == "kg/d":
        # Daily load conversion requires volume flow data
        #raise ValueError("Conversion for 'kg/d' requires daily volume flow data.")
        return value, unit
    elif unit == "kg/qtr":
        # Quarterly load conversion requires volume flow data
        #raise ValueError("Conversion for 'kg/qtr' requires quarterly volume flow data.")
        return value, unit
    elif unit == "kt/qtr":
        # Kilotons per quarter conversion requires volume flow data
        #raise ValueError("Conversion for 'kt/qtr' requires quarterly volume flow data.")
        return value, unit
    elif unit == "ug/kg":
        # Assuming water density, 1 ug/kg = 0.001 mg/l
        return value * 0.001, cUnit
    elif unit == "ng/l":
        # 1 ng/l = 1e-6 mg/l
        return value * 1e-6, cUnit
    elif unit == "no/100ml":
        # Conversion depends on the context of measurement
        #raise ValueError("Conversion for 'no/100ml' requires context-specific information.")
        return value, unit
    elif unit == "us/cm":
        # Conductivity conversion needs additional information about the substance
        #raise ValueError("Conversion for 'us/cm' requires substance-specific details.")
        return value, unit
    elif unit == "no/g":
        # Conversion for counts per gram needs context
        #raise ValueError("Conversion for 'no/g' requires context-specific information.")
        return value, unit
    elif unit == "ppm":
        # 1 ppm = 1 mg/l (assuming water density)
        return value * 1, cUnit
    elif unit == "t/ann":
        # Tons per annum conversion requires flow data
        #raise ValueError("Conversion for 't/ann' requires annual volume flow data.")
        return value, unit
    elif unit == "t/d":
        # Tons per day conversion requires flow data
        #raise ValueError("Conversion for 't/d' requires daily volume flow data.")
        return value, unit
    elif unit == "t/qtr":
        # Tons per quarter conversion requires flow data
        #raise ValueError("Conversion for 't/qtr' requires quarterly volume flow data.")
        return value, unit
    elif unit == "t/wk":
        # Tons per week conversion requires flow data
        #raise ValueError("Conversion for 't/wk' requires weekly volume flow data.")
        return value, unit
    elif unit == "ug":
        # Micrograms conversion needs volume data
        #raise ValueError("Conversion for 'ug' requires volume data.")
        return value, unit
    elif unit == "ng/kg":
        # Assuming water density, 1 ng/kg = 1e-6 mg/l
        return value * 1e-6, cUnit
    elif unit == "pg/l":
        # 1 pg/l = 1e-9 mg/l
        return value * 1e-9, cUnit
    elif unit == "pg/m3":
        # Conversion requires volume information
        #raise ValueError("Conversion for 'pg/m3' requires volume context.")
        return value, unit
    elif unit == "rfu":
        # Relative fluorescence units are context-specific
        #raise ValueError("Conversion for 'rfu' is context-specific and requires calibration data.")
        return value, unit
    else:
        #raise ValueError("Unknown unit: {}".format(unit))
        return value, unit

# Example usage
try:
    result, resultUnit = first_convert_to_mg_per_l(5, "%")
    print("Converted value: {} {}".format(result, resultUnit))
except ValueError as e:
    print("Error:", e)


Converted value: 50000 mg/l


In [12]:
# Metrics Conversion functions
def second_convert_to_mg_per_l(value, unit):
    """
    Converts various metrics to mg/L.
    Only applicable units are converted; others raise a ValueError.
    """
    unit = unit.lower()
    conversion_factors = {
        "mg/l": 1,            # Already in mg/L
        "%": 10_000,          # Assumes % w/v (1% = 10,000 mg/L)
        "ug/l": 1e-3,         # Micrograms per liter
        "mg/kg": 1,           # Assuming 1 kg/L density
        "% v/v": 10_000,      # Assumes % v/v in water
        "% w/w": 10_000,      # Assumes % w/w in water
        "g/l": 1_000,         # Grams per liter
        "ug/kg": 1e-3,        # Micrograms per kg (assuming 1 kg/L density)
        "ng/l": 1e-6,         # Nanograms per liter
        "ppm": 1,             # Equivalent to mg/L
        "bq/l": 1,            # Assumes radionuclide mass equivalent in mg/L
        "cfu/0.1l": 10,       # Colony-forming units per 0.1L to mg/L (arbitrary mass equivalence)
        "kg/d": 1 / 0.001,    # Assuming flow of 1 L/d
        "kg/qtr": 1 / (0.001 * 90),  # Quarterly conversion with 90 days assumed
        "kt/qtr": 1e6 / (0.001 * 90),# Kilotons per quarter
        "no/100ml": 10,       # Assuming 1 unit mass equivalent
        "us/cm": None,        # Cannot convert electrical conductivity to mg/L directly
        "no/g": None,         # Cannot convert counts to mg/L directly
        "t/d": 1e6 / 0.001,   # Tons per day assuming 1 L/d
        "t/qtr": 1e6 / (0.001 * 90),
        "t/wk": 1e6 / (0.001 * 7),
        "coded": None,        # Unsupported
        "m": None,            # Unsupported
        "ngr": None,          # Unsupported
        "unitless": None,     # Unsupported
        "phi": None,          # Unsupported
        "min": None,          # Unsupported
        "deg": None,          # Unsupported
        "deccafix": None,     # Unsupported
        "pres/nf": None,      # Unsupported
        "mw.s/sq.": None,     # Unsupported
        "no.": None,          # Unsupported
        "abs/cm": None,       # Unsupported
        "nm": None,           # Unsupported
        "text": None,         # Unsupported
        "yes/no": None,       # Unsupported
        "l/kg/h": None,       # Unsupported
        "ug": 1e-3,           # Micrograms to mg
    }

    if unit in conversion_factors and conversion_factors[unit] is not None:
        return value * conversion_factors[unit]
    else:
        #raise ValueError(f"Unsupported or non-convertible unit: {unit}")
        #raise ValueError(f"Unsupported")
        raise ValueError(np.nan)


# Example usage
try:
    #User input
    #value = float(input("Enter the value: "))
    #unit = input("Enter the unit: ")
    
    #Hardcoded values
    value = 1.1
    unit = 'ug'
    unit = 'text'
    result = second_convert_to_mg_per_l(value, unit)
    print(f"{value} {unit} is equal to {result} mg/L.")
except ValueError as e:
    print(e)


nan


In [13]:
'''
    Principal Component Analysis (PCA) reduces dimensionality by transforming a large dataset with many variables into a smaller set of 
    principal components. These components capture the most important variance, aiding visualization, analysis, and model simplification.

    What it does:
    - Reduces dimensionality while preserving original variance.
    - Transforms correlated variables into uncorrelated principal components.
    - Highlights variation to identify patterns and relationships.

    How it works:
    - Standardize the data to mean 0 and standard deviation 1.
    - Calculate the covariance matrix.
    - Find eigenvectors and eigenvalues.
    - Select principal components with the largest eigenvalues.
    - Transform the data by projecting onto principal components.

    Why use it?
    - Data visualization: Easier exploration and understanding.
    - Exploratory data analysis: Identifies patterns in large datasets.
    - Machine learning: Improves performance and reduces computational complexity.
    - Data compression: Reduces storage while preserving essential information.
'''

'''
    PCA Definition
'''

'\n    PCA Definition\n'

In [14]:
#Function to find PCA
def find_optimal_pca_components(X, explained_variance_threshold=0.90):
    """
    Finds the optimal number of PCA components to reach the desired cumulative explained variance.

    Parameters:
    - X: np.ndarray, The input data matrix (n_samples, n_features).
    - explained_variance_threshold: float, The target cumulative explained variance ratio (default is 0.90).
    
    Returns:
    - optimal_components: int, Optimal number of components to reach the target explained variance.
    - cumulative_explained_variance: np.ndarray, Cumulative explained variance for each component.
    """
    # Standardize the data
    X = StandardScaler().fit_transform(X)

    # Fit PCA without specifying the number of components
    pca = PCA().fit(X)

    # Calculate cumulative explained variance
    cumulative_explained_variance = np.cumsum(pca.explained_variance_ratio_)

    # Determine the optimal number of components
    optimal_components = np.argmax(cumulative_explained_variance >= explained_variance_threshold) + 1
    
    # Plot cumulative explained variance
    plt.figure(figsize=(8, 5))
    plt.plot(range(1, len(cumulative_explained_variance) + 1), cumulative_explained_variance, marker='o', linestyle='--')
    plt.xlabel('Number of Components')
    plt.ylabel('Cumulative Explained Variance')
    plt.title('Explained Variance vs. Number of Components')
    plt.axhline(y=explained_variance_threshold, color='r', linestyle='-')
    plt.grid()
    plt.show()
    
    print(f"Optimal number of components to reach {explained_variance_threshold*100:.0f}% explained variance: {optimal_components}")
    
    return optimal_components, cumulative_explained_variance


In [15]:
#Revised Function to find PCA
def find_optimal_pca_components_new(X, explained_variance_threshold=0.90):
    # Standardize the data
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    # Apply PCA
    #pca = PCA(n_components=0.95)  # Keep 95% of variance
    pca = PCA(explained_variance_threshold)  # Keep 95% of variance
    
    X_pca = pca.fit_transform(X_scaled)

    # Get feature names for principal components
    def get_component_names(pca, feature_names, n_top_features=3):
        component_names = []
        for i, component in enumerate(pca.components_):
            # Get indices of top contributing features
            top_idx = np.argsort(np.abs(component))[::-1][:n_top_features]
            # Get names and coefficients
            top_features = [f"{feature_names[j]} ({component[j]:.2f})"
                            for j in top_idx]
            name = f"PC{i+1} [" + ", ".join(top_features) + "]"
            component_names.append(name)
        return component_names
    
    # Get reduced feature names
    reduced_features = get_component_names(pca, X.columns)
    
    # Create DataFrame with reduced features
    df_pca = pd.DataFrame(X_pca, columns=reduced_features)

    #print("Reduced feature names with top contributors:")
    #print("\n".join(reduced_features))
    #print("\nReduced feature dataset preview:")
    #print(df_pca.head())
    print(f"\nOriginal features: {X.shape[1]}")
    print(f"Reduced features: {len(reduced_features)}")
    print(f"Variance explained: {np.sum(pca.explained_variance_ratio_):.2%}")
    

    # Optional: Feature selection using SelectKBest
    from sklearn.feature_selection import SelectKBest, f_classif
    selector = SelectKBest(f_classif, k=5)
    X_selected = selector.fit_transform(X, y)
    selected_features = X.columns[selector.get_support()]
    print("\nSelected features:", list(selected_features))
    
    return df_pca, selected_features


In [16]:
# Read Whole 24 Years Transposed dataset [Below conditional statements ensure the data to be 
# loaded is either With or without Reactive monitoring observations]
folderpath = 'cleansed/'

'''
Commented on 12 May 2025
filename = '12_NB_10Transpose_2959_All2_With_R_Mon1.csv' #Total records: 3711376 | Bigger size [All types of Sample purpose Including Reactive monitoring] 
#filename = '12_NB_10Transpose_2959_All2_Without_R_Mon1.csv' #Total records: 3642495 | Smaller size [All types of Sample purpose Except Reactive monitoring]
'''

filename = '08_nb_transpose_2959_all2_with_r_monLess_UL_C.csv' #Added on 12 May 2025
filename = '08_nb_transpose_2959_all2_with_r_mon.csv' #Added on 14 May 2025

showtime()
dffull = loaddata(folderpath, filename, path = 'gcs://rdmai_dev_data/')
showtime()
print(len(dffull))


15 May 2025 21:28:37
gcs://rdmai_dev_data/cleansed/08_nb_transpose_2959_all2_with_r_mon.csv
15 May 2025 21:53:13
9278441


In [17]:
'''
optimal_feat_with_r_mon = ['sampleDateOnly','determinand_unit_name_label','isComplianceSample_label','purpose_name_label',
                           'samplingPoint_easting','samplingPoint_northing','117','118','119','9924','241','237','158',
                           '211','192','76','301','9901','207','463','77','6450','6455','7859','1183','1181','82',
                           '6528','163','6529','6452','6530','6460','3410','52','6531','3408','108','3409','7887','50','6462',
                           '3164','106','164','6051','460','186','6458','105','6045','2348','6050','68',
                           '180','192','348']
dffull[optimal_feat_with_r_mon].head(2)
#KeyError: "['3695', '6426', '6428', '2353', '2355', '2364', '9398'] not in index"
'''

'\noptimal_feat_with_r_mon = [\'sampleDateOnly\',\'determinand_unit_name_label\',\'isComplianceSample_label\',\'purpose_name_label\',\n                           \'samplingPoint_easting\',\'samplingPoint_northing\',\'117\',\'118\',\'119\',\'9924\',\'241\',\'237\',\'158\',\n                           \'211\',\'192\',\'76\',\'301\',\'9901\',\'207\',\'463\',\'77\',\'6450\',\'6455\',\'7859\',\'1183\',\'1181\',\'82\',\n                           \'6528\',\'163\',\'6529\',\'6452\',\'6530\',\'6460\',\'3410\',\'52\',\'6531\',\'3408\',\'108\',\'3409\',\'7887\',\'50\',\'6462\',\n                           \'3164\',\'106\',\'164\',\'6051\',\'460\',\'186\',\'6458\',\'105\',\'6045\',\'2348\',\'6050\',\'68\',\n                           \'180\',\'192\',\'348\']\ndffull[optimal_feat_with_r_mon].head(2)\n#KeyError: "[\'3695\', \'6426\', \'6428\', \'2353\', \'2355\', \'2364\', \'9398\'] not in index"\n'

In [18]:
#Filter the Date out of Datetime
dffull['sampleDateOnly'] = pd.to_datetime(dffull['sampleDateOnly'])
dffull['sampleDateOnly'] = dffull['sampleDateOnly'].dt.date
dffull = dffull.set_index('sampleDateOnly')
dffull.head(2)

,samplingPoint_notation,samplingPoint_easting,samplingPoint_northing,determinand_unit_name_label,purpose_name_label,isComplianceSample_label,3,4,6,28,30,31,37,39,50,52,61,62,64,66,67,68,69,72,73,76,77,81,82,83,85,88,97,99,103,105,106,108,111,113,114,116,117,118,119,134,135,138,143,144,152,153,158,162,163,164,167,171,172,174,175,177,179,180,182,183,186,191,192,205,207,209,211,235,237,239,241,254,270,301,346,348,460,461,462,463,483,487,491,495,499,503,507,511,527,531,535,539,543,547,551,555,559,562,569,570,573,575,576,577,578,579,581,612,664,666,714,723,729,731,733,736,738,739,746,749,753,758,772,831,911,912,947,950,979,1002,1049,1066,1085,1109,1115,1118,1119,1120,1153,1158,1173,1181,1183,1193,1198,1200,2331,2348,2551,2896,2921,2922,2923,2924,2925,2926,2942,2943,2944,2953,2955,2957,2958,2959,2960,2961,2962,2963,2964,2965,2967,2968,2969,2970,2972,2973,2975,2977,2981,2982,2983,2984,2987,2988,2989,2990,2991,2993,2995,3000,3001,3002,3004,3007,...,8940,8942,8943,8944,8994,8995,8997,8998,8999,9000,9002,9003,9005,9007,9009,9011,9013,9015,9017,9019,9021,9035,9039,9040,9050,9051,9052,9066,9068,9073,9091,9094,9097,9142,9143,9145,9156,9160,9161,9166,9190,9196,9197,9198,9199,9238,9239,9248,9258,9260,9267,9274,9276,9285,9338,9339,9340,9341,9342,9343,9344,9345,9348,9350,9364,9366,9371,9446,9447,9451,9453,9454,9456,9457,9458,9460,9461,9462,9466,9467,9468,9472,9474,9475,9477,9479,9494,9496,9519,9522,9552,9573,9584,9586,9603,9604,9606,9618,9619,9634,9638,9639,9669,9671,9684,9686,9688,9691,9695,9696,9703,9715,9716,9724,9725,9726,9727,9728,9729,9730,9731,9732,9733,9734,9735,9736,9737,9738,9739,9740,9741,9742,9743,9744,9745,9746,9747,9748,9749,9750,9751,9756,9768,9769,9770,9771,9772,9773,9774,9807,9809,9811,9814,9815,9816,9817,9818,9819,9821,9822,9823,9836,9839,9841,9844,9845,9851,9853,9856,9857,9860,9861,9862,9863,9880,9883,9885,9888,9889,9891,9892,9898,9899,9901,9911,9921,9924,9925,9933,9935,9943,9951,9952,9959,9978,9979,9989,9990,9992,9993
sampleDateOnly,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2000-01-01,AN-70561033,489852,289360,28,24,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,8.70,0.00,0.00,0.00,0.00,13.00,0.00,0.00,2.10,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,110.00,0.00,0.00,0.00,0.00,0.00,0.00,740.00,0.00,0.00,0.00,0.00,260.00,0.00,21.00,0.00,54.00,0.00,300.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.05,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00

In [19]:
#Find CUMULATIVE Features (Reduced Features) using PCA - Begins

In [20]:
### Method 1: Using datasets with a stratified re-sampling method
#.  (For large data, this step filters out the rows proportionately from a representative sample of each distinguishable sampling point pattern)
print("Total Rows of the full Dataset   : ", len(dffull))
#dffull_sample = dffull.sample(frac=0.5, random_state=1).reset_index(drop=False) 
dffull_sample = dffull.sample(frac=0.1, random_state=1).reset_index(drop=False)
print("Total Rows of the sampled Dataset: ", len(dffull_sample))
dffull_sample.head(2)
#dffull_sample = dffull.copy() #As the stratified sample was already taken during Transpose2 at 12_NB_10Transpose_2959_All2.ipynb

#3642495  9278441
#1821248  1855688

Total Rows of the full Dataset   :  9278441
Total Rows of the sampled Dataset:  927844


,sampleDateOnly,samplingPoint_notation,samplingPoint_easting,samplingPoint_northing,determinand_unit_name_label,purpose_name_label,isComplianceSample_label,3,4,6,28,30,31,37,39,50,52,61,62,64,66,67,68,69,72,73,76,77,81,82,83,85,88,97,99,103,105,106,108,111,113,114,116,117,118,119,134,135,138,143,144,152,153,158,162,163,164,167,171,172,174,175,177,179,180,182,183,186,191,192,205,207,209,211,235,237,239,241,254,270,301,346,348,460,461,462,463,483,487,491,495,499,503,507,511,527,531,535,539,543,547,551,555,559,562,569,570,573,575,576,577,578,579,581,612,664,666,714,723,729,731,733,736,738,739,746,749,753,758,772,831,911,912,947,950,979,1002,1049,1066,1085,1109,1115,1118,1119,1120,1153,1158,1173,1181,1183,1193,1198,1200,2331,2348,2551,2896,2921,2922,2923,2924,2925,2926,2942,2943,2944,2953,2955,2957,2958,2959,2960,2961,2962,2963,2964,2965,2967,2968,2969,2970,2972,2973,2975,2977,2981,2982,2983,2984,2987,2988,2989,2990,2991,2993,2995,3000,3001,3002,3004,...,8940,8942,8943,8944,8994,8995,8997,8998,8999,9000,9002,9003,9005,9007,9009,9011,9013,9015,9017,9019,9021,9035,9039,9040,9050,9051,9052,9066,9068,9073,9091,9094,9097,9142,9143,9145,9156,9160,9161,9166,9190,9196,9197,9198,9199,9238,9239,9248,9258,9260,9267,9274,9276,9285,9338,9339,9340,9341,9342,9343,9344,9345,9348,9350,9364,9366,9371,9446,9447,9451,9453,9454,9456,9457,9458,9460,9461,9462,9466,9467,9468,9472,9474,9475,9477,9479,9494,9496,9519,9522,9552,9573,9584,9586,9603,9604,9606,9618,9619,9634,9638,9639,9669,9671,9684,9686,9688,9691,9695,9696,9703,9715,9716,9724,9725,9726,9727,9728,9729,9730,9731,9732,9733,9734,9735,9736,9737,9738,9739,9740,9741,9742,9743,9744,9745,9746,9747,9748,9749,9750,9751,9756,9768,9769,9770,9771,9772,9773,9774,9807,9809,9811,9814,9815,9816,9817,9818,9819,9821,9822,9823,9836,9839,9841,9844,9845,9851,9853,9856,9857,9860,9861,9862,9863,9880,9883,9885,9888,9889,9891,9892,9898,9899,9901,9911,9921,9924,9925,9933,9935,9943,9951,9952,9959,9978,9979,9989,9990,9992,9993
0,2006-08-17,MD-26949580,363085,319459,57,3,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,2013-05-09,MD-9001

In [21]:
dffull_sample['sampleDateOnly'].isna().sum()

0

In [22]:
#dffull_tempPCA = dffull.head(15000000)
#X = dffull_sample.drop(columns=['180', 'samplingPoint_notation', 'sampleDateOnly'])  # Replace 'OrthoP' with your target column
X = dffull_sample.drop(columns=['180', 'samplingPoint_notation', 'sampleDateOnly'])  # Dropped sampleDateOnly to avoid TypeError: float() argument must be a string or a real number, not 'datetime.date'
#X = X.set_index(['sampleDateOnly'])
y = dffull_sample['180']

df_pca, selected_features = find_optimal_pca_components_new(X, explained_variance_threshold=0.98)
print (df_pca.columns)
df_pca.head(2)


Original features: 970
Reduced features: 520
Variance explained: 98.03%

Selected features: ['85', '301', '4365', '4366', '9924']
Index(['PC1 [487 (0.13), 562 (0.13), 569 (0.13)]',
       'PC2 [6744 (0.17), 6541 (0.17), 6658 (0.17)]',
       'PC3 [2967 (0.18), 2955 (0.18), 2991 (0.18)]',
       'PC4 [4436 (0.14), 4438 (0.14), 4442 (0.14)]',
       'PC5 [4346 (0.15), 4345 (0.15), 4349 (0.15)]',
       'PC6 [9730 (0.20), 9729 (0.20), 9742 (0.20)]',
       'PC7 [6649 (0.17), 9860 (0.17), 3399 (0.17)]',
       'PC8 [9716 (0.19), 507 (0.19), 9715 (0.19)]',
       'PC9 [3636 (0.25), 3635 (0.25), 3646 (0.25)]',
       'PC10 [6657 (0.26), 6613 (0.26), 6573 (0.25)]',
       ...
       'PC511 [6534 (0.41), 6532 (-0.40), 6057 (-0.25)]',
       'PC512 [6057 (0.39), 6450 (-0.36), 162 (-0.33)]',
       'PC513 [162 (0.44), 9924 (-0.43), 6057 (0.39)]',
       'PC514 [7859 (0.65), 205 (-0.58), 9857 (-0.22)]',
       'PC515 [6971 (0.52), 9267 (-0.25), 8997 (-0.24)]',
       'PC516 [4826 (0.69), 4824 (-

,"PC1 [487 (0.13), 562 (0.13), 569 (0.13)]","PC2 [6744 (0.17), 6541 (0.17), 6658 (0.17)]","PC3 [2967 (0.18), 2955 (0.18), 2991 (0.18)]","PC4 [4436 (0.14), 4438 (0.14), 4442 (0.14)]","PC5 [4346 (0.15), 4345 (0.15), 4349 (0.15)]","PC6 [9730 (0.20), 9729 (0.20), 9742 (0.20)]","PC7 [6649 (0.17), 9860 (0.17), 3399 (0.17)]","PC8 [9716 (0.19), 507 (0.19), 9715 (0.19)]","PC9 [3636 (0.25), 3635 (0.25), 3646 (0.25)]","PC10 [6657 (0.26), 6613 (0.26), 6573 (0.25)]","PC11 [8311 (0.27), 8360 (0.27), 736 (0.27)]","PC12 [5681 (0.27), 5680 (0.27), 5283 (0.27)]","PC13 [9456 (0.24), 4117 (0.23), 9458 (0.23)]","PC14 [8049 (0.22), 8050 (0.21), 8356 (0.21)]","PC15 [8890 (0.32), 8892 (0.32), 8888 (0.32)]","PC16 [5042 (0.25), 8806 (0.22), 8807 (0.22)]","PC17 [3282 (0.34), 9836 (0.34), 7531 (0.33)]","PC18 [5044 (0.28), 5045 (0.27), 5048 (0.27)]","PC19 [6455 (0.25), 6450 (0.25), 3408 (0.25)]","PC20 [5160 (0.34), 5152 (0.34), 5153 (0.34)]","PC21 [4854 (0.29), 5891 (0.29), 9019 (0.28)]","PC22 [6987 (0.24), 6972 (0.24), 6955 (0.24)]","PC23 [3555 (0.31), 3791 (0.30), 9068 (0.30)]","PC24 [9911 (0.37), 6635 (0.36), 5037 (0.36)]","PC25 [7077 (0.42), 7072 (0.42), 7075 (0.42)]","PC26 [7395 (0.30), 8940 (0.30), 9003 (0.30)]","PC27 [7395 (0.27), 9003 (0.27), 8940 (0.27)]","PC28 [4143 (0.38), 4144 (0.38), 4142 (0.38)]","PC29 [6776 (0.43), 9951 (0.42), 9035 (0.41)]","PC30 [3817 (0.35), 3812 (0.33), 3778 (0.32)]","PC31 [6603 (0.34), 5157 (0.34), 7086 (0.34)]","PC32 [6530 (0.31), 1181 (0.30), 6531 (0.29)]","PC33 [9460 (0.23), 3290 (0.23), 6530 (-0.22)]","PC34 [3958 (0.40), 3957 (0.38), 3962 (0.37)]","PC35 [73 (0.50), 7562 (0.50), 9341 (0.50)]","PC36 [9166 (0.16), 5731 (0.16), 9460 (0.15)]","PC37 [8052 (0.42), 9741 (0.41), 9731 (0.39)]","PC38 [9021 (0.34), 9007 (0.34), 9009 (0.33)]","PC39 [237 (0.33), 207 (0.32), 211 (0.30)]","PC40 [9166 (0.41), 5731 (0.40), 5733 (0.37)]","PC41 [3268 (0.50), 3373 (0.50), 3328 (0.50)]","PC42 [2959 (0.30), 2964 (0.27), 2960 (0.27)]","PC43 [9883 (0.33), 3792 (0.33), 6620 (0.28)]","PC44 [5562 (0.32), 9522 (0.26), 6940 (0.25)]","PC45 [3792 (0.28), 9883 (0.28), 6620 (0.24)]","PC46 [4352 (0.31), 4366 (-0.29), 4365 (-0.29)]","PC47 [6516 (0.50), 6517 (0.46), 8296 (0.46)]","PC48 [9021 (0.23), 9007 (0.23), 9009 (0.23)]","PC49 [5291 (0.31), 9952 (0.27), 8162 (0.22)]","PC50 [3458 (0.48), 2551 (0.43), 2331 (0.41)]","PC51 [9338 (0.39), 9344 (0.39), 9343 (0.36)]","PC52 [9943 (0.46), 6485 (0.44), 9853 (0.36)]","PC53 [4755 (0.45), 4869 (0.43), 4754 (0.38)]","PC54 [7136 (0.29), 7137 (0.25), 6830 (0.24)]","PC55 [6693 (0.20), 9446 (0.20), 6980 (-0.17)]","PC56 [3905 (0.32), 3906 (0.28), 3904 (0.24)]","PC57 [753 (0.31), 575 (0.31), 9618 (-0.29)]","PC58 [3703 (0.60), 3537 (0.57), 3533 (0.56)]","PC59 [575 (0.32), 753 (0.32), 5732 (0.23)]","PC60 [9724 (0.36), 9728 (0.35), 9732 (0.34)]","PC61 [9924 (0.34), 117 (0.33), 162 (0.31)]","PC62 [575 (0.28), 753 (0.28), 6062 (0.24)]","PC63 [5923 (0.59), 4906 (0.57), 4907 (0.52)]","PC64 [9143 (0.34), 7071 (0.27), 1109 (0.27)]","PC65 [9143 (0.31), 6633 (0.26), 1109 (0.24)]","PC66 [9453 (0.50), 9454 (0.49), 3341 (0.35)]","PC67 [4822 (0.34), 106 (0.32), 6064 (-0.32)]","PC68 [7859 (0.50), 205 (0.50), 235 (0.38)]","PC69 [7755 (0.62), 7756 (0.56), 6374 (0.52)]","PC70 [9889 (0.65), 9898 (0.65), 9899 (0.36)]","PC71 [3000 (0.28), 6665 (0.28), 9040 (-0.22)]","PC72 [5948 (0.35), 5951 (0.32), 3026 (-0.23)]","PC73 [5948 (0.29), 5951 (0.27), 3549 (-0.24)]","PC74 [3026 (0.39), 3027 (0.39), 241 (0.28)]","PC75 [9040 (0.44), 9039 (0.43), 3000 (0.42)]","PC76 [3027 (0.41), 3026 (0.41), 114 (0.25)]","PC77 [6665 (0.42), 3000 (0.42), 9040 (-0.27)]","PC78 [7173 (0.69), 7174 (0.69), 3463 (-0.10)]","PC79 [3960 (0.49), 3965 (-0.46), 3954 (0.45)]","PC80 [3463 (0.68), 3462 (0.68), 7174 (0.10)]","PC81 [6559 (0.24), 3127 (0.23), 7202 (-0.22)]","PC82 [4826 (0.62), 4824 (0.49), 4827 (0.47)]","PC83 [6559 (0.26), 3790 (0.21), 6591 (0.19)]","PC84 [9161 (0.71), 8046 (0.71), 8288 (0.01)]","PC85 [3820 (0.48), 3819 (0.47), 3818 (0.45)]","PC86 [3164 (0.53), 340

In [23]:
selected_features

Index(['85', '301', '4365', '4366', '9924'], dtype='object')

In [24]:
df_pca.columns

Index(['PC1 [487 (0.13), 562 (0.13), 569 (0.13)]',
       'PC2 [6744 (0.17), 6541 (0.17), 6658 (0.17)]',
       'PC3 [2967 (0.18), 2955 (0.18), 2991 (0.18)]',
       'PC4 [4436 (0.14), 4438 (0.14), 4442 (0.14)]',
       'PC5 [4346 (0.15), 4345 (0.15), 4349 (0.15)]',
       'PC6 [9730 (0.20), 9729 (0.20), 9742 (0.20)]',
       'PC7 [6649 (0.17), 9860 (0.17), 3399 (0.17)]',
       'PC8 [9716 (0.19), 507 (0.19), 9715 (0.19)]',
       'PC9 [3636 (0.25), 3635 (0.25), 3646 (0.25)]',
       'PC10 [6657 (0.26), 6613 (0.26), 6573 (0.25)]',
       ...
       'PC511 [6534 (0.41), 6532 (-0.40), 6057 (-0.25)]',
       'PC512 [6057 (0.39), 6450 (-0.36), 162 (-0.33)]',
       'PC513 [162 (0.44), 9924 (-0.43), 6057 (0.39)]',
       'PC514 [7859 (0.65), 205 (-0.58), 9857 (-0.22)]',
       'PC515 [6971 (0.52), 9267 (-0.25), 8997 (-0.24)]',
       'PC516 [4826 (0.69), 4824 (-0.57), 4823 (-0.44)]',
       'PC517 [2925 (0.80), 2984 (0.20), 8162 (0.19)]',
       'PC518 [8721 (0.47), 7924 (-0.26), 6633 (-0.2

In [25]:
print(filename)

08_nb_transpose_2959_all2_with_r_mon.csv


In [26]:
#Save PCA dataset
#df_pca.to_csv('../NW_DataPP/14_NB_6Correlation_2959_All_PCA.csv', index=True)
#savedata(df_pca, "14_NB_6Correlation_2959_With_R_Mon1_PCA.csv", tpath='gcs://rdmai_dev_data/NW_Final/')

'''
if (filename == "08_nb_transpose_2959_all2_with_r_monLess_UL_C.csv"):
    #savedata(df_pca, "14_NB_6Correlation_2959_with_r_mon_PCA.csv", tpath='gcs://rdmai_dev_data/NW_Final/')
    savedata(df_pca, "09_nb_correlation_2959_all_with_r_mon_PCA.csv", tpath='gcs://rdmai_dev_data/NW_Final/')
else:
    savedata(df_pca, "09_nb_correlation_2959_all_without_r_mon_PCA.csv", tpath='gcs://rdmai_dev_data/NW_Final/')
'''

if (filename == "08_nb_transpose_2959_all2_with_r_mon.csv"):
    savedata(df_pca, "09_nb_correlation_2959_all_with_r_mon_PCA.csv", tpath='gcs://rdmai_dev_data/')
else:
    savedata(df_pca, "09_nb_correlation_2959_all_without_r_mon_PCA.csv", tpath='gcs://rdmai_dev_data/')

del (df_pca, X, y, dffull_sample)

saved, Location:  gcs://rdmai_dev_data/cleansed/09_nb_correlation_2959_all_with_r_mon_PCA.csv


In [27]:
#Find CUMULATIVE Features using PCA - Ends

In [28]:
#Find Correlation Coefficients - Begins

In [29]:
# calculate the Pearson's correlation between two variables
from numpy.random import randn
from numpy.random import seed
from scipy.stats import pearsonr
# seed random number generator
seed(1)

# prepare data
#cols = dffull.columns

#print('Pearsons correlation for the determinands')
#X = StandardScaler().fit_transform(dffull[cols[2:]])
#print(type(X), X.shape)

# Method 2: Using dataframe's `sample` method (For Large Data)
#dffull_sample = dffull.sample(frac=0.2, random_state=1).reset_index(drop=False)
dffull_sample = dffull.sample(frac=0.15, random_state=1).reset_index(drop=False)
print(len(dffull))
print(len(dffull_sample))
dffull_sample.head(2)

9278441
1391766


,sampleDateOnly,samplingPoint_notation,samplingPoint_easting,samplingPoint_northing,determinand_unit_name_label,purpose_name_label,isComplianceSample_label,3,4,6,28,30,31,37,39,50,52,61,62,64,66,67,68,69,72,73,76,77,81,82,83,85,88,97,99,103,105,106,108,111,113,114,116,117,118,119,134,135,138,143,144,152,153,158,162,163,164,167,171,172,174,175,177,179,180,182,183,186,191,192,205,207,209,211,235,237,239,241,254,270,301,346,348,460,461,462,463,483,487,491,495,499,503,507,511,527,531,535,539,543,547,551,555,559,562,569,570,573,575,576,577,578,579,581,612,664,666,714,723,729,731,733,736,738,739,746,749,753,758,772,831,911,912,947,950,979,1002,1049,1066,1085,1109,1115,1118,1119,1120,1153,1158,1173,1181,1183,1193,1198,1200,2331,2348,2551,2896,2921,2922,2923,2924,2925,2926,2942,2943,2944,2953,2955,2957,2958,2959,2960,2961,2962,2963,2964,2965,2967,2968,2969,2970,2972,2973,2975,2977,2981,2982,2983,2984,2987,2988,2989,2990,2991,2993,2995,3000,3001,3002,3004,...,8940,8942,8943,8944,8994,8995,8997,8998,8999,9000,9002,9003,9005,9007,9009,9011,9013,9015,9017,9019,9021,9035,9039,9040,9050,9051,9052,9066,9068,9073,9091,9094,9097,9142,9143,9145,9156,9160,9161,9166,9190,9196,9197,9198,9199,9238,9239,9248,9258,9260,9267,9274,9276,9285,9338,9339,9340,9341,9342,9343,9344,9345,9348,9350,9364,9366,9371,9446,9447,9451,9453,9454,9456,9457,9458,9460,9461,9462,9466,9467,9468,9472,9474,9475,9477,9479,9494,9496,9519,9522,9552,9573,9584,9586,9603,9604,9606,9618,9619,9634,9638,9639,9669,9671,9684,9686,9688,9691,9695,9696,9703,9715,9716,9724,9725,9726,9727,9728,9729,9730,9731,9732,9733,9734,9735,9736,9737,9738,9739,9740,9741,9742,9743,9744,9745,9746,9747,9748,9749,9750,9751,9756,9768,9769,9770,9771,9772,9773,9774,9807,9809,9811,9814,9815,9816,9817,9818,9819,9821,9822,9823,9836,9839,9841,9844,9845,9851,9853,9856,9857,9860,9861,9862,9863,9880,9883,9885,9888,9889,9891,9892,9898,9899,9901,9911,9921,9924,9925,9933,9935,9943,9951,9952,9959,9978,9979,9989,9990,9992,9993
0,2006-08-17,MD-26949580,363085,319459,57,3,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,2013-05-09,MD-9001

In [30]:
# Example usage with any n-dimensional data
# Replace `your_data` with your actual n-dimensional data matrix
# your_data = np.array(...)  # Replace this line with your data loading code
# For illustration, let's generate random data:
#np.random.seed(42)
#your_data = np.random.rand(100, 10)  # Example data with 100 samples and 10 features

'''
colselect = dffull.columns

#dfPt_192_Cr.iloc[:, 1:].head(5)
optimal_components, cumulative_explained_variance = find_optimal_pca_components(
                                                    dffull_sample[colselect[2:]], explained_variance_threshold=0.98)

print(optimal_components, cumulative_explained_variance.size)
'''

'\ncolselect = dffull.columns\n\n#dfPt_192_Cr.iloc[:, 1:].head(5)\noptimal_components, cumulative_explained_variance = find_optimal_pca_components(\n                                                    dffull_sample[colselect[2:]], explained_variance_threshold=0.98)\n\nprint(optimal_components, cumulative_explained_variance.size)\n'

In [31]:
#! pip install cupy-cuda12x
import cupy as cp
#! pip install polars
import polars as pl

colselect = dffull.columns


In [34]:
len(dffull)

#9278441
#3642495

In [32]:
showtime()
'''
dffull_sample = dffull[colselect[2:]]
dffull_pl = pl.from_pandas(dffull_sample)
test_180 = dffull_pl.corr()["180"]
test_180 = test_180.rename("pearson_180")
showtime()
test_180.head(2)
'''
del(dffull)
gc.collect()

15 May 2025 22:09:38


132

In [35]:
# Final output - Pearson Linear Correlation Coefficient
# Calculate correlation coefficients of each variable with the target
#180	Orthophospht	Orthophosphate, reactive as P	1506334
showtime()
dffull_sample = dffull_sample[colselect[2:]]
Pearson_correlations_180 = dffull_sample.corr(method='pearson', min_periods=1, numeric_only=False)["180"].drop("180")


# Display the correlations
print("Pearson correlations_180 Coefficients ----Completed----:")
#print(Pearson_correlations_180.sort_values(ascending=False))
Pearson_correlations_180 = Pearson_correlations_180.rename("pearson_180")
showtime()
Pearson_correlations_180.head(2)

15 May 2025 22:10:45
Pearson correlations_180 Coefficients ----Completed----:
15 May 2025 23:00:59


samplingPoint_northing        -0.01
determinand_unit_name_label   -0.01
Name: pearson_180, dtype: float64

In [36]:
#savedata(Pearson_correlations_180, "14_NB_6Correlation_2959_Pear_Spear_Without_R_Mon1.csv", tpath='gcs://rdmai_dev_data/NW_Final/') #Commented on 11 May 2025


In [ ]:
# Final output - Spearman’s rank correlation coefficient
# Calculate correlation coefficients of each variable with the target
#180	Orthophospht	Orthophosphate, reactive as P	1506334
#dffull_sample = dffull[colselect[2:]]
Spearman_correlations_180 = dffull_sample.corr(method='spearman', min_periods=1, numeric_only=False)["180"].drop("180")
Spearman_correlations_180 = Spearman_correlations_180.rename("spearman_180")

# Display the correlations
print("Spearman correlations_192 Coefficients ----Completed----:")
#print(Spearman_correlations_192.sort_values(ascending=False))
Spearman_correlations_180.head(2)

Spearman correlations_192 Coefficients ----Completed----:


samplingPoint_northing        -0.00
determinand_unit_name_label   -0.06
Name: spearman_180, dtype: float64

In [ ]:
#Pearson_correlations_180 = Pearson_correlations_180.rename("pearson_180")
#Pearson_correlations_180[0:5]

In [ ]:
print(type(Spearman_correlations_180),type(Pearson_correlations_180))
res = []
res = pd.concat([Pearson_correlations_180, Spearman_correlations_180], axis=1)
print(len(res))
del(Pearson_correlations_180, Spearman_correlations_180)
gc.collect()
res.head(2)
#Spearman_correlations -It is rank based rather than values. Relationship is Monotonic; Move at random pace.
#Pearson_correlations - It is value based. Relationship is Linear. Move at same pace.

<class 'pandas.core.series.Series'> <class 'pandas.core.series.Series'>
969


,pearson_180,spearman_180
samplingPoint_northing,-0.01,-0.00
determinand_unit_name_label,-0.01,-0.06


In [ ]:
def func_optimal_feats(pearson_180, spearman_180):
    pearson = abs(pearson_180)
    spearman = abs(spearman_180)
    if pearson == 0:
        res =  spaerman;
    elif pearson > spearman:
        res = pearson;
    elif pearson < spearman:
        res = spearman
    else:
        res = 0
    return res

res['optimal_feats'] = res.apply(lambda x: func_optimal_feats(x['pearson_180'], x['spearman_180']), axis=1)
#res.sort_values(by="optimal_feats", ascending=False)
print("Find bigger - Step completed")

Find bigger - Step completed


In [41]:
#print(res.query("optimal_feats >= 0.3").reset_index().agg(list, 0)) #Pasu to be continued

In [44]:
#clean memory
del (dffull_sample, Spearman_correlations_180)
gc.collect()

In [43]:
# Save the Transposed data - Full data Transposed
#res.to_csv('../NW_DataPP/14_NB_6Correlation_2959_Pear_Spear_With_R_Mon1.csv', index=True)
#res.to_csv('../NW_DataPP/14_NB_6Correlation_2959_Pear_Spear_Without_R_Mon1.csv', index=True)
#savedata(res, "14_NB_6Correlation_2959_Pear_Spear_With_R_Mon1.csv", tpath='gcs://rdmai_dev_data/NW_Final/')
#savedata(res, "14_NB_6Correlation_2959_Pear_Spear_Without_R_Mon1.csv", tpath='gcs://rdmai_dev_data/NW_Final/') #Commented on 15 May 2025
#savedata(res, "09_nb_correlation_2959_all_Pear_Spear_with_r_mon.csv", tpath='gcs://rdmai_dev_data/')  #Commented on 15 May 2025

if (filename == "08_nb_transpose_2959_all2_with_r_mon.csv"):
    savedata(res, "09_nb_correlation_2959_all_Pear_Spear_with_r_mon.csv", tpath='gcs://rdmai_dev_data/')  #Added on 15 May 2025
else:
    savedata(res, "09_nb_correlation_2959_all_Pear_Spear_without_r_mon.csv", tpath='gcs://rdmai_dev_data/')  #Added on 15 May 2025


saved, Location:  gcs://rdmai_dev_data/cleansed/09_nb_correlation_2959_all_Pear_Spear_with_r_mon.csv


In [45]:
#END CARD

In [46]:
res.head(2)

,pearson_180,spearman_180,optimal_feats
samplingPoint_northing,-0.01,-0.00,0.01
determinand_unit_name_label,-0.01,-0.06,0.06
